# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn.metrics import mean_absolute_error,mean_squared_error,  r2_score
from pycaret.regression import *
from pycaret.utils import check_metric

# Load Data

In [19]:
panCancer_train_df = pd.read_csv('../../D2GNets/data/panCancer_train_CGC_657_DR_Drug_features_new_df.csv')
GBM_train_df = pd.read_csv('../../D2GNets/data/GBM_train_CGC_657_DR_Drug_features_new_df.csv')

# Feature Engineeing (Step-2)

In [20]:
# Removing Features based on Feature Score less than 70
featureScore = pd.read_csv('../../D2GNets/supplementary_material/featureScore.csv')
featureScore = featureScore[featureScore['Score']>=70]
features = set(featureScore['Name'].to_list())

In [22]:
# Getting the column names
disease_cols = (pd.read_csv('../../D2GNets/data/diseaseNames.csv')).columns.tolist()
GE_cols =  (pd.read_csv('../../D2GNets/data/657_Gene_name.csv')).columns.tolist()
drug_cols =  (pd.read_csv('../../D2GNets/data/moleculeNames_v1.csv')).columns.tolist()

Drug_Xcols = list(features.intersection(set(drug_cols)))+disease_cols
GE_Xcols = GE_cols#list(features.intersection(set(panCancer_train_df.columns[55+2986:].to_list())))

In [23]:
print(f"No. of Diseases: {len(disease_cols)}")
print(f"No. of Drugs Features: {len(Drug_Xcols)}")
print(f"No. of GENEs: {len(GE_Xcols)}")

No. of Diseases: 31
No. of Drugs Features: 1999
No. of GENEs: 657


# Data Preparation for ML operation

### Data for Drug Repurposing Experiments

In [24]:
GBM_test_Carmustine = GBM_train_df[GBM_train_df['DRUG_NAME']=='Carmustine'].copy(deep=True)
GBM_test_Temozolomide = GBM_train_df[GBM_train_df['DRUG_NAME']=='Temozolomide'].copy(deep=True)

### Data for Drug Screening Experiments

In [25]:
indx_4_drugs = (GBM_test_Carmustine.index.to_list()+
 GBM_test_Temozolomide.index.to_list())
GBM_train_df.drop(index=indx_4_drugs,inplace=True)

In [26]:
panCancer_df = pd.concat([panCancer_train_df,GBM_train_df],axis=0)
panCancer_df = panCancer_df[Drug_Xcols+GE_Xcols+['TCGA_DESC','LN_IC50','SAMPLE_ID','DRUG_NAME']]
panCancer_df.dropna(inplace=True)

In [27]:
train = panCancer_df.sample(frac=0.8).copy(deep=True)# Training data for Drug Screening
test = panCancer_df.drop(index=train.index).copy(deep=True) # Test Data for Drug Screening

In [28]:
print(f"Train data size: {train.shape[0]}")
print(f"Test data size: {test.shape[0]}")

Train data size: 96219
Test data size: 22845


# Quantile Transformation of Target (LN_IC50)

In [29]:
qt1 = QuantileTransformer(n_quantiles=2, random_state=42,output_distribution='uniform')
#qt2 = QuantileTransformer(n_quantiles=200, random_state=0)

train['LN_IC50_t'] = qt1.fit_transform(
    train['LN_IC50'].values.reshape(-1, 1))
test['LN_IC50_t'] = qt1.fit_transform(
    test['LN_IC50'].values.reshape(-1, 1))

#test_unseen['LN_IC50_t'] = qt1.fit_transform(test_unseen['LN_IC50'].values.reshape(-1, 1))

GBM_test_Carmustine['LN_IC50_t'] = qt1.fit_transform(GBM_test_Carmustine['LN_IC50'].values.reshape(-1, 1))
GBM_test_Temozolomide['LN_IC50_t'] = qt1.fit_transform(GBM_test_Temozolomide['LN_IC50'].values.reshape(-1, 1))

# Random Forest Model for Quantile Inverse Transformation

In [31]:
exp1 = setup(data=train[['LN_IC50_t','LN_IC50']], target='LN_IC50', test_data=test[['LN_IC50_t','LN_IC50']],numeric_features=['LN_IC50_t']
           ,transformation = False, normalize=False,use_gpu=True,polynomial_features=True,polynomial_degree=3)

,Description,Value
0,session_id,930
1,Target,LN_IC50
2,Original Data,"(96219, 2)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(96219, 2)"


In [32]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0240
huber,Huber Regressor,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.4550
knn,K Neighbors Regressor,0.0001,0.0000,0.0015,1.0000,0.0002,0.0002,1.3570
omp,Orthogonal Matching Pursuit,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0080
br,Bayesian Ridge,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0300
lar,Least Angle Regression,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0080
rf,Random Forest Regressor,0.0001,0.0000,0.0011,1.0000,0.0001,0.0001,4.6760
dt,Decision Tree Regressor,0.0001,0.0000,0.0012,1.0000,0.0001,0.0001,0.2410
ridge,Ridge Regression,0.0031,0.0000,0.0043,1.0000,0.0009,0.0028,0.0070
et,Extra Trees Regressor,0.0000,0.0000,0.0007,1.0000,0.0001,0.0000,5.4740


In [33]:
et_LNIC50 = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0000,0.0000,0.0001,1.0000,0.0000,0.0000
1,0.0001,0.0000,0.0044,1.0000,0.0004,0.0000
2,0.0000,0.0000,0.0001,1.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
4,0.0000,0.0000,0.0001,1.0000,0.0000,0.0000
5,0.0000,0.0000,0.0016,1.0000,0.0002,0.0000
6,0.0000,0.0000,0.0001,1.0000,0.0000,0.0000
7,0.0000,0.0000,0.0003,1.0000,0.0000,0.0000
8,0.0000,0.0000,0.0001,1.0000,0.0000,0.0000
9,0.0000,0.0000,0.0001,1.0000,0.0000,0.0000


In [34]:
et_res= pull()

In [38]:
et_res.to_csv('../../D2GNets/result/LabelModel10fold.csv')

In [36]:
predict_model(et_LNIC50);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0719,0.0068,0.0823,0.9991,0.0327,0.1459


In [37]:
final_et_LNIC50 = finalize_model(et_LNIC50)

In [39]:
save_model(final_et_LNIC50,'../../D2GNets/models/LNIC50_Model/Final_et_LNIC50_Model_25June2022')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['LN_IC50_t'],
                                       target='LN_IC50', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numer...
                  ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                      criterion='mse', max_depth=None,
                                      max_features='auto', max_leaf_nodes=None,
                                      max_samples=None,
                                    

# Drug Screening Model development

In [24]:
exp = setup(data=train, target='LN_IC50_t', test_data=test,  ignore_features=['TCGA_DESC','LN_IC50','SAMPLE_ID','DRUG_NAME'],numeric_features=disease_cols+Drug_Xcols+GE_Xcols
           ,transformation = False, normalize=False,use_gpu=True)

,Description,Value
0,session_id,6778
1,Target,LN_IC50_t
2,Original Data,"(96219, 2661)"
3,Missing Values,False
4,Numeric Features,2656
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(96219, 1313)"


In [27]:
models()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


In [25]:
mdl1= compare_models(include=['lr','knn','gbr','dt','ridge','ada','lasso','rf','lightgbm','et'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0473,0.0040,0.0631,0.8271,0.0414,0.0993,15.5250
rf,Random Forest Regressor,0.0484,0.0043,0.0653,0.8151,0.0428,0.1005,524.7860
et,Extra Trees Regressor,0.0482,0.0043,0.0653,0.8148,0.0430,0.0997,323.4130
ridge,Ridge Regression,0.0502,0.0045,0.0672,0.8043,0.0441,0.1056,4.7370
lr,Linear Regression,0.0503,0.0045,0.0673,0.8034,0.0442,0.1058,3.6330
gbr,Gradient Boosting Regressor,0.0602,0.0061,0.0780,0.7365,0.0512,0.1300,713.5630
dt,Decision Tree Regressor,0.0688,0.0087,0.0930,0.6247,0.0607,0.1382,72.6800
knn,K Neighbors Regressor,0.0840,0.0133,0.1153,0.4236,0.0762,0.1934,211.9050
ada,AdaBoost Regressor,0.0982,0.0138,0.1173,0.4035,0.0757,0.1999,474.5170
lasso,Lasso Regression,0.1178,0.0231,0.1519,-0.0001,0.1009,0.2888,0.9800


In [ ]:
#mdl1_1 = pull()
#mdl1_1.to_csv('../../D2GNets/result/modelSelection.csv')

In [65]:
lr = create_model('lr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0502,0.0045,0.0671,0.7994,0.0440,0.1071
1,0.0499,0.0045,0.0668,0.8075,0.0439,0.1056
2,0.0497,0.0044,0.0665,0.8080,0.0435,0.1025
3,0.0507,0.0046,0.0678,0.8034,0.0445,0.1071
4,0.0498,0.0044,0.0664,0.8111,0.0435,0.1028
5,0.0510,0.0047,0.0686,0.7976,0.0450,0.1057
6,0.0505,0.0046,0.0678,0.8010,0.0445,0.1070
7,0.0496,0.0044,0.0663,0.8090,0.0436,0.1079
8,0.0499,0.0044,0.0662,0.8093,0.0435,0.1037
9,0.0497,0.0044,0.0665,0.8078,0.0437,0.1052


In [29]:
lgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0474,0.0040,0.0634,0.8280,0.0415,0.0974
1,0.0475,0.0041,0.0638,0.8323,0.0421,0.1036
2,0.0476,0.0041,0.0637,0.8207,0.0416,0.0969
3,0.0474,0.0039,0.0628,0.8292,0.0413,0.1032
4,0.0475,0.0040,0.0633,0.8240,0.0415,0.0984
5,0.0475,0.0040,0.0634,0.8252,0.0417,0.1034
6,0.0471,0.0040,0.0629,0.8245,0.0411,0.0977
7,0.0475,0.0040,0.0634,0.8249,0.0417,0.0990
8,0.0470,0.0039,0.0627,0.8283,0.0411,0.0970
9,0.0470,0.0039,0.0624,0.8288,0.0407,0.0960


In [36]:
x=pd.DataFrame({'Feature': get_config('X_train').columns}).to_csv('../../D2GNets/result/feature.csv')

In [67]:
lgbm_res= pull()

In [68]:
lgbm_res.to_csv('../../D2GNets/result/LGBM10fold.csv')

In [69]:
final_lgbm = finalize_model(lgbm)
save_model(final_et_LNIC50,'../../D2GNets/models/LGBM/Final_lgbm_Model_25June2022')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True,
                                       features_todrop=['TCGA_DESC', 'LN_IC50',
                                                        'SAMPLE_ID',
                                                        'DRUG_NAME'],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['ACC', 'ALL', 'BLCA',
                                                           'BRCA', 'CESC', 'CLL',
                                                           'COREAD', 'DLBC',
                                                           'ESCA', 'GBM', 'HNSC',
                                                           'KIRC', 'LAML',
                                                           'LCML', 'LGG', 'LIHC',
                                                 

# Drug Screening experiment with Test data

In [70]:
# Prediction
unseen_predict =predict_model(lgbm, data=test)

In [ ]:
#saving the resutlt
unseen_predict.to_csv('../../D2GNets/result/unseen_predict.csv',index=False)

# Drug Repurposing Experiment 

In [72]:
# Prediction 
GBM_Carmustine_pred = predict_model(final_lgbm, data=GBM_test_Carmustine)
GBM_Temozolomide_pred = predict_model(final_lgbm, data=GBM_test_Temozolomide)

In [73]:
# Saving the Results of drug Repurposing
GBM_Carmustine_pred.to_csv('../../D2GNets/result/GBM_Carmustine_pred.csv',index=False)
GBM_Temozolomide_pred.to_csv('../../D2GNets/result/GBM_Temozolomide_pred.csv',index=False)